In [2]:
import torch

In [3]:
X = torch.randn((10,3)) 
Y = torch.Tensor([x.sum()>.5 for x in X]).reshape(-1,1)

In [4]:
X, Y

(tensor([[-1.5393,  0.3881, -0.6518],
         [-0.4476,  1.6374, -0.3800],
         [-0.5326, -0.5929,  0.0245],
         [ 0.8944,  0.6916, -0.1333],
         [-1.2887,  0.1412,  0.1543],
         [ 0.4261,  1.4891,  0.5128],
         [ 0.8890, -1.1942,  0.1213],
         [ 0.4749,  2.3916,  0.3019],
         [ 0.6243, -0.1385, -0.7694],
         [ 0.3668,  0.5391, -1.1731]]),
 tensor([[0.],
         [1.],
         [0.],
         [1.],
         [0.],
         [1.],
         [0.],
         [1.],
         [0.],
         [0.]]))

In [5]:
W1 = torch.randn(size=(3,5))
b1 = torch.randn(size=(1,))
W2 = torch.randn(size=(5,5))
b2 = torch.randn(size=(1,))
W3 = torch.randn(size=(5,1))
b3 = torch.randn(size=(1,))

In [6]:
W1, b1

(tensor([[-0.2956, -0.9351, -0.1666,  1.5829,  0.2097],
         [ 0.6204, -0.1358, -0.7459, -2.0544, -0.3974],
         [-0.1034, -1.2711, -0.5991, -0.1179, -0.2275]]),
 tensor([-1.5624]))

In [7]:
W2, b2

(tensor([[ 1.6657, -0.3510,  0.0906,  0.3588,  0.3337],
         [-0.6926, -1.6872,  0.1447,  1.0868,  0.5247],
         [ 0.4417,  0.0696,  0.0018, -0.0593, -0.1147],
         [ 0.9833, -0.4152, -0.9829, -0.9597,  0.6316],
         [-0.6366,  0.0140,  0.4750, -1.6092,  1.2523]]),
 tensor([-1.4533]))

In [8]:
W3, b3

(tensor([[-1.3029],
         [ 0.7900],
         [-0.6873],
         [ 0.4082],
         [ 0.2608]]),
 tensor([-0.1504]))

In [9]:
z1 = X@W1+b1
h1 = torch.tanh(z1)
z2 = h1@W2+b2
h2 = torch.tanh(z2)
z3 = h2@W3+b3
probs = torch.sigmoid(z3)
Y_hat = torch.Tensor([1 if a > .5 else 0 for a in probs],).unsqueeze(1)

In [10]:
acc = (Y_hat == Y).sum() / len(Y)
loss = -probs.log().mean()
acc, loss

(tensor(0.4000), tensor(0.2014))

In [17]:
dloss_dprobs = (1/len(Y)) * (1/probs)
dprobs_z3 =  probs * (1-probs)
dz3_h2 = W3
dz3_W3 = h2
dh2_z2 = h2 * (1-h2)
dz2_dh1 = W2
dz2_dW2 = h1
dh1_z1 = h1 * (1-h1)
dz1_dW1 = X

In [69]:
dloss_dW3 = (dz3_W3 * dloss_dprobs * dprobs_z3).mean(0).unsqueeze(dim=1)
dloss_dW2 = (dloss_dprobs * dprobs_z3).mean(0) * dz3_h2 * (dh2_z2 * dz2_dW2).mean(0)
dloss_dW1 = (dloss_dprobs * dprobs_z3).mean(0) * dz3_h2 * dh2_z2.mean(0) * dz2_dh1 * dh1_z1.mean(0)
dloss_dW1 = X.sum(0).unsqueeze(dim=1).broadcast_to(size=(3,5)) @ dloss_dW1

In [70]:
W3, dloss_dW3

(tensor([[-1.3029],
         [ 0.7900],
         [-0.6873],
         [ 0.4082],
         [ 0.2608]]),
 tensor([[-0.0176],
         [ 0.0017],
         [-0.0152],
         [-0.0019],
         [-0.0180]]))

In [71]:
W2, dloss_dW2

(tensor([[ 1.6657, -0.3510,  0.0906,  0.3588,  0.3337],
         [-0.6926, -1.6872,  0.1447,  1.0868,  0.5247],
         [ 0.4417,  0.0696,  0.0018, -0.0593, -0.1147],
         [ 0.9833, -0.4152, -0.9829, -0.9597,  0.6316],
         [-0.6366,  0.0140,  0.4750, -1.6092,  1.2523]]),
 tensor([[-0.0334,  0.0054, -0.0319,  0.0027, -0.0429],
         [ 0.0202, -0.0032,  0.0193, -0.0016,  0.0260],
         [-0.0176,  0.0028, -0.0168,  0.0014, -0.0226],
         [ 0.0105, -0.0017,  0.0100, -0.0008,  0.0134],
         [ 0.0067, -0.0011,  0.0064, -0.0005,  0.0086]]))

In [73]:
W1, dloss_dW1

(tensor([[-0.2956, -0.9351, -0.1666,  1.5829,  0.2097],
         [ 0.6204, -0.1358, -0.7459, -2.0544, -0.3974],
         [-0.1034, -1.2711, -0.5991, -0.1179, -0.2275]]),
 tensor([[ 1.7212e-02,  1.8017e-04,  1.7539e-03,  5.6591e-04, -5.3913e-03],
         [-6.9413e-01, -7.2658e-03, -7.0734e-02, -2.2822e-02,  2.1742e-01],
         [ 2.5846e-01,  2.7054e-03,  2.6337e-02,  8.4978e-03, -8.0957e-02]]))